# Auto Fairness Demo

End-to-end walkthrough: generate synthetic auto underwriting data, train GLM / NN / ADV_NN, and evaluate fairness at both default and fixed approval thresholds.


## 1. Setup


In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / "src").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"Using project root: {PROJECT_ROOT}")


In [ ]:
from __future__ import annotations

import numpy as np
import pandas as pd
import torch
from dataclasses import replace
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

from src.auto.data_generation_auto import generate_auto_underwriting_data, train_test_split_auto
from src.config import TrainingConfig
from src.evaluation.metrics import compute_accuracy_metrics
from src.evaluation.fairness import fairness_metrics, fairness_at_target_rate
from src.models.glm_model import GLMClassifier
from src.models.nn_model import PlainNN, train_plain_nn, predict_proba_plain_nn
from src.models.adv_nn_model import AdvPredictor, train_adv_nn, predict_proba_adv_nn


## 2. Generate Auto Data


In [ ]:
df = generate_auto_underwriting_data(n_samples=100_000, seed=123)
print(f"Dataset size: {len(df):,}")
df.head()


## 3. Train/Test Split


In [ ]:
df_train, df_test = train_test_split_auto(df, seed=123)
print(f"Train size: {len(df_train):,}, Test size: {len(df_test):,}")


### Feature preprocessing


In [ ]:
feature_cols_cont = ["Age", "M_h", "V"]
binary_cols = ["P_prior", "T"]

scaler = StandardScaler()
X_train_cont = scaler.fit_transform(df_train[feature_cols_cont])
X_test_cont = scaler.transform(df_test[feature_cols_cont])

X_train = np.concatenate(
    [X_train_cont, df_train[binary_cols].to_numpy()],
    axis=1
).astype(np.float32)
X_test = np.concatenate(
    [X_test_cont, df_test[binary_cols].to_numpy()],
    axis=1
).astype(np.float32)

y_train = df_train["Y"].to_numpy(dtype=np.float32)
y_test = df_test["Y"].to_numpy(dtype=np.float32)
A_train = (df_train["Race"] == "A").astype(int).to_numpy(dtype=np.int64)
A_test = (df_test["Race"] == "A").astype(int).to_numpy(dtype=np.int64)

youth_train = (df_train["Age"] < 25).astype(int).to_numpy(dtype=np.float32)
youth_test = (df_test["Age"] < 25).astype(int).to_numpy(dtype=np.float32)
X_train_glm = np.concatenate([X_train, youth_train[:, None]], axis=1)
X_test_glm = np.concatenate([X_test, youth_test[:, None]], axis=1)

train_cfg = TrainingConfig()

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=123, stratify=y_train
)

def build_loader(X, y, batch_size, shuffle=True):
    ds = TensorDataset(torch.from_numpy(X).float(), torch.from_numpy(y).float())
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle)

train_loader = build_loader(X_tr, y_tr, train_cfg.batch_size, shuffle=True)
val_loader = build_loader(X_val, y_val, train_cfg.batch_size, shuffle=False)
test_loader = build_loader(X_test, y_test, train_cfg.batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train.shape, X_test.shape, device


## 4. GLM Baseline


In [ ]:
glm = GLMClassifier().fit(X_train_glm, y_train)
y_proba_glm = glm.predict_proba(X_test_glm)


## 5. Neural Network


In [ ]:
plain_nn = PlainNN(input_dim=X_train.shape[1]).to(device)
train_plain_nn(plain_nn, train_loader, val_loader, train_cfg, device)
y_proba_nn = predict_proba_plain_nn(plain_nn, X_test, device=device)


## 6. Adversarial Neural Network


In [ ]:
train_cfg_adv = replace(train_cfg, lambda_adv=0.8)

adv_ds = TensorDataset(
    torch.from_numpy(X_train).float(),
    torch.from_numpy(y_train).float(),
    torch.from_numpy(A_train).long(),
)
adv_loader = DataLoader(adv_ds, batch_size=train_cfg.batch_size, shuffle=True)

adv_model = AdvPredictor(input_dim=X_train.shape[1]).to(device)
train_adv_nn(adv_model, adv_loader, train_cfg_adv, device=device)
y_proba_adv = predict_proba_adv_nn(adv_model, X_test, device=device)


## 7. Evaluation & Fairness


In [ ]:
def summarize(model, y_true, y_proba, A_true, threshold=0.5):
    acc = compute_accuracy_metrics(y_true, y_proba)
    fair = fairness_metrics(y_true, y_proba, A_true, threshold=threshold)
    return {"model": model, **acc, **fair}

summary_default = pd.DataFrame(
    [
        summarize("GLM", y_test, y_proba_glm, A_test),
        summarize("NN", y_test, y_proba_nn, A_test),
        summarize("ADV_NN", y_test, y_proba_adv, A_test),
    ]
)
summary_default


### Target 0.02

In [ ]:
TARGET_RATE = 0.02

def summarize_fixed(model, y_true, y_proba, A_true):
    fair = fairness_at_target_rate(y_true, y_proba, A_true, TARGET_RATE)
    acc = compute_accuracy_metrics(y_true, y_proba)
    return {"model": model, **acc, **fair}

summary_fixed = pd.DataFrame(
    [
        summarize_fixed("GLM", y_test, y_proba_glm, A_test),
        summarize_fixed("NN", y_test, y_proba_nn, A_test),
        summarize_fixed("ADV_NN", y_test, y_proba_adv, A_test),
    ]
)
summary_fixed[
    ["model", "roc_auc", "eo_gap_tpr", "eo_gap_fpr", "dp_diff", "dp_ratio", "threshold", "actual_rate"]
]


## 8. Summary


In [ ]:
text = (
    "### Key Insights\n"
    f"- GLM ROC AUC {summary_fixed.loc[summary_fixed.model=='GLM', 'roc_auc'].iat[0]:.3f}; "
    f"EO gap {summary_fixed.loc[summary_fixed.model=='GLM', 'eo_gap_tpr'].iat[0]:.3f}; "
    f"DP ratio {summary_fixed.loc[summary_fixed.model=='GLM', 'dp_ratio'].iat[0]:.2f} at 2% approval.\n"
    f"- NN ROC AUC {summary_fixed.loc[summary_fixed.model=='NN', 'roc_auc'].iat[0]:.3f}; "
    f"EO gap {summary_fixed.loc[summary_fixed.model=='NN', 'eo_gap_tpr'].iat[0]:.3f}; "
    f"DP ratio {summary_fixed.loc[summary_fixed.model=='NN', 'dp_ratio'].iat[0]:.2f} with modest fairness changes.\n"
    f"- ADV_NN ROC AUC {summary_fixed.loc[summary_fixed.model=='ADV_NN', 'roc_auc'].iat[0]:.3f}; "
    f"EO gap {summary_fixed.loc[summary_fixed.model=='ADV_NN', 'eo_gap_tpr'].iat[0]:.3f}; "
    f"DP ratio {summary_fixed.loc[summary_fixed.model=='ADV_NN', 'dp_ratio'].iat[0]:.2f} reflecting adversarial mitigation.\n"
)
print(text)


## 9. Auto experiment plots

Use the auto bias sweep outputs to visualize fairness vs accuracy and fairness vs injected bias strength. Run `src/experiments/auto/bias_sweep.py` first to populate `results/auto/*/bias_sweep_metrics.csv`.


In [ ]:
from src.experiments.auto import plot_fairness_accuracy_frontier as auto_frontier
from src.experiments.auto import plot_fairness_vs_rate as auto_vs_rate
from IPython.display import Image, display
from pathlib import Path


### 9.1 Fairness vs accuracy frontier

Scatter EO gap vs ROC AUC across bias strengths for each model.


In [ ]:
auto_frontier_path = auto_frontier.main()
display(Image(filename=auto_frontier_path))


### 9.2 Fairness vs bias strength

Plot DP ratio and EO gap as the injected bias strength varies.


In [ ]:
dp_bias_path, eo_bias_path = auto_vs_rate.main()
display(Image(filename=dp_bias_path))
display(Image(filename=eo_bias_path))
